## Evalutate model


In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D, InputLayer
from tensorflow.keras import Sequential
import matplotlib.pyplot as plt
import os

## Restore model

In [ ]:
def build_CNN_2_model(settings):
    # this has 34,826 training parameters -> ~411kB
    k1 = settings['conv2d_1']['kernel_size']
    n1 = settings['conv2d_1']['n_nodes']
    k2 = settings['conv2d_2']['kernel_size']
    n2 = settings['conv2d_2']['n_nodes']
    model = Sequential(
        [
            InputLayer(input_shape=(28, 28, 1)),
            Conv2D(n1, kernel_size=(k1, k1), activation="relu"),
            MaxPooling2D(pool_size=(2, 2)),
            Conv2D(n2, kernel_size=(k2, k2), activation="relu"),
            MaxPooling2D(pool_size=(2, 2)),
            Flatten(),
            Dropout(0.5),
            Dense(10, activation="softmax"),
        ]
    )
    return model

settings = {
        "CNN_2": { # 34,826 parameters
            'conv2d_1': {
                'kernel_size': 3,
                'n_nodes': 32
            },
            'conv2d_2': {
                'kernel_size': 3,
                'n_nodes': 64
            }, 
        },
        "CNN_3": { # 12,810 parameters
            'conv2d_1': {
                'kernel_size': 3,
                'n_nodes': 16
            },
            'conv2d_2': {
                'kernel_size': 3,
                'n_nodes': 32
            },
        },
         "CNN_4": { # 18,378 parameters
            'conv2d_1': {
                'kernel_size': 5,
                'n_nodes': 16
            },
            'conv2d_2': {
                'kernel_size': 5,
                'n_nodes': 32
            }, 
        },
    }

def build_NN_model():
    model = Sequential()
    model.add(Flatten(input_shape=(28, 28, 1)))
    model.add(Dense(32, activation='sigmoid'))
    model.add(Dense(10, activation='softmax'))
    return model

In [ ]:
checkpoint_dir = 'models/CNN_4_74k/'
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

In [ ]:
model = build_CNN_2_model(settings['CNN_4'])
#model = build_NN_model()
model.load_weights(latest)
print("weights restored!")

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
dummy_pic = np.random.random((28, 28))
plt.imshow(dummy_pic, cmap='gray', vmin=0, vmax=1);

In [ ]:
dummy_pic = dummy_pic.reshape(1, 28, 28, 1)
np.argmax(model.predict(dummy_pic))

## Load data

In [ ]:
def load_mnist(mnist_path):
    labels = []
    digits = []
    with open(mnist_path) as f:
        f.readline()
        for line in f.readlines():
            data = line.split(",")
            label = int(data[0])
            img = np.array(list(map(int, data[1:]))).reshape((28, 28))
            labels.append(label)
            digits.append(img)
    labels = np.array(labels)
    digits = np.array(digits)
    return digits, labels

In [ ]:
input_dir = "../datasets"
dataset = "74k" # MNIST font 74k combined
train_path = os.path.join(input_dir, dataset + "_train.csv")
test_path = os.path.join(input_dir, dataset + "_test.csv")

In [ ]:
print("loading data ...")
train_digits, train_labels = load_mnist(train_path)
test_digits, test_labels = load_mnist(test_path)
print("data loaded.")

## Prep for NN

In [ ]:
x_train = train_digits.astype('float32')/255
y_train = tf.keras.utils.to_categorical(train_labels, num_classes=10)

y_test = tf.keras.utils.to_categorical(test_labels, num_classes=10)
x_test = test_digits.astype('float32')/255

x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)

## Evaluate

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=2)
print("test evaluation")
print("accuracy: {:5.2f}%".format(100 * acc))
print("loss:     {:5.2f}".format(loss))

In [ ]:
loss, acc = model.evaluate(x_train, y_train, verbose=2)
print("test evaluation")
print("accuracy: {:5.2f}%".format(100 * acc))
print("loss:     {:5.2f}".format(loss))

### Confusion Matrix

In [ ]:
y_pred = np.array([np.argmax(y) for y in model.predict(x_test, verbose=2)])

In [ ]:
y_labels = np.array([np.argmax(y) for y in y_test])

In [ ]:
cm = tf.math.confusion_matrix(y_pred, y_labels, num_classes=10)
cm

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
ax.imshow(cm)
# Loop over data dimensions and create text annotations.
for i in range(len(cm)):
    for j in range(len(cm[0])):
        text = ax.text(j, i, cm[i, j].numpy(), ha="center", va="center", color="w")
ax.set_xlabel('predicted classes')
ax.set_ylabel('real classes')
ax.set_xticks(range(10));
ax.set_yticks(range(10));

### Plot incorrect predictions

In [ ]:
x_data = x_test

In [ ]:
nrow = 3
fig, axes = plt.subplots(nrow, 10, figsize=(12, 5))
for j in range(10):
    idxs_wrong = np.logical_and(y_pred == j, y_pred != y_labels)
    idxs_j = np.arange(len(y_pred))[idxs_wrong]
    labels_j = y_pred[idxs_wrong]
    idxs = np.random.choice(range(len(labels_j)), size=min(len(labels_j), nrow), replace=False)
    for i in range(min(len(labels_j), nrow)):
        ax = axes[i][j]
        ax.imshow(x_data[idxs_j[i]], cmap='gray', vmin=0, vmax=1)
        #ax.axis('off')
        ax.set_xlabel(y_labels[idxs_j[i]])
        ax.get_xaxis().set_ticklabels([])
        ax.get_yaxis().set_ticklabels([])
    axes[0][j].set_title(str(j))

## Plot intermediate layers - Conv

In [ ]:
layer_idx = 0
intermediate_model = tf.keras.Model(
    inputs=model.input,
    outputs=model.layers[layer_idx].output
    )

In [ ]:
intermediate_model.summary()

In [ ]:
idx = 1800
plt.imshow(x_train[idx]);
plt.title("label: {:}".format(train_labels[idx]));

Plot features for this input

In [ ]:
features = intermediate_model.predict(np.array([x_train[idx]]))

In [ ]:
num_features = features.shape[-1]
ncol = 10;
nrow = int(np.ceil(num_features/ncol))

fig, axes = plt.subplots(nrow, ncol, figsize=(14, num_features/5))
for idx in range(num_features):
    i, j = idx//ncol, idx % ncol
    ax = axes[i][j]
    ax.axis('off')
    ax.imshow(features[0, :, :, idx])
for idx in range(num_features, ncol*nrow):
    i, j = idx//ncol, idx % ncol
    ax = axes[i][j]
    ax.axis('off')
axes[0][int(ncol//2)-1].set_title("{:d} features for layer {:d}".format(num_features, layer_idx));

plot kernels

In [ ]:
n_kernels = model.layers[layer_idx].kernel.shape[-1]
ncol = 10;
nrow = int(np.ceil(num_features/ncol))

fig, axes = plt.subplots(nrow, ncol, figsize=(14, n_kernels/5))
for idx in range(n_kernels):
    i, j = idx//ncol, idx % ncol
    ax = axes[i][j]
    ax.axis('off')
    kernel = model.layers[layer_idx].kernel[:, :, :, idx]
    ax.imshow(kernel)
for idx in range(n_kernels, ncol*nrow):
    i, j = idx//ncol, idx % ncol
    ax = axes[i][j]
    ax.axis('off')

## Plot intermediate layers - Dense

In [ ]:
layer_idx = 6
num_features = 32
n_slice = 1
input_shape = (4, 4, 32, 10) # (4, 4, 32, 10)  (28, 28, 32)
weights = model.layers[layer_idx].weights[0].numpy().reshape(input_shape)
biases =  model.layers[layer_idx].weights[1]

ncol = 10;
nrow = int(np.ceil(num_features/ncol))
fig, axes = plt.subplots(nrow, ncol, figsize=(14, num_features/5))
for idx in range(num_features):
    i, j = idx//ncol, idx % ncol
    ax = axes[i][j]
    ax.axis('off')
    ax.imshow(weights[:, :, idx, n_slice])
for idx in range(num_features, ncol*nrow):
    i, j = idx//ncol, idx % ncol
    ax = axes[i][j]
    ax.axis('off')